# Lab Exercise 5: QAM-PSK

```{image} ../content/images/QAM16_Demonstration.gif
:align: center
```

```{admonition} Live Code
Press the following button to make python code interactive. It will connect you to a kernel once it says "ready" (might take a bit, especially the first time it runs).
```

<div style="text-align: center;">
  <button title="Launch thebe" class="thebelab-button thebe-launch-button" onclick="initThebe()">Python Interactive Code</button>
</div>


#### Importing packages we will need later in Python


In [3]:
# Import necessary libraries for numerical computations and plotting
import numpy as np
import matplotlib.pyplot as plt

# Import interactive widgets for creating interactive elements in Jupyter notebooks
import ipywidgets as widgets
from ipywidgets import interact, fixed, interact_manual
from ipywidgets import IntSlider, IntRangeSlider, FloatSlider, interactive, Layout, Dropdown, IntText, HBox, VBox, Output

# Import display functions for Jupyter notebooks
from IPython.display import display, clear_output

# Import additional libraries for signal processing and mathematical operations
import random
import scipy.signal
from math import log
import array
from scipy.signal import upfirdn, welch, decimate, convolve
from scipy.special import erfc
import time

# Confirm that the libraries have been successfully imported
print("Libraries added successfully!")

Libraries added successfully!


## Part 1: QAM constellation plot
### Question 1
Plot a 64-QAM signal constellation of a complete rectangular grid, with the binary words next to each of its points marked [as in figure 5.4(b) of the Course Notes] with Gray coding.

```{hint}
Use the section of commands 26-34 from Code 5.2 of the Course Notes (repeated below,
for convenience) to produce a mapping vector containing all points of the signal constellation
in ascending codeword order: mapping(1) -> 00..00, mapping (2)->00..01, … With the
scatterplot() command, plot the points of the signal constellation and with appropriate
text addition commands write near each point the corresponding code word [e.g. the
command text(3,3,num2str(de2bi(4,3,'left-msb')), 'FontSize', 6); writes
the codeword “1 0 0” at point (3,3) of the figure with font size 6].
```

``````{dropdown} Constellation Code
`````{tab} Python
````python
# The mapping vector for M-QAM Gray coding
# It is for a complete rectangular constellation, of dimension M=L^2
# l=log2(L): number of bits per dimension (inphase, quadrature)
core = np.array([1+1j, 1-1j, -1+1j, -1-1j]) # trivial case, M=4
mapping = core

if (l > 1):
    for k in range (1,l):
        mapping = mapping + k*2*core[0]
        mapping = np.array([mapping, np.conj(mapping)])
        mapping = np.array([mapping, (-np.conj(mapping))])
        
mapping = mapping.flatten().T

````
`````
`````{tab} Matlab
````matlab

% The mapping vector for M-QAM Gray coding
% It is for a complete rectangular constellation, of dimension M=L^2
% l=log2(L): number of bits per dimension (inphase, quadrature)
core=[1+i;1-i;-1+i;-1-i]; % trivial case, M=4
mapping=core;
if(l>1)
 for j=1:l-1
 mapping=mapping+j*2*core(1);
 mapping=[mapping;conj(mapping)];
 mapping=[mapping;-conj(mapping)];
 end
end;


````
`````
``````

In [5]:
# Loading animation HTML code for visual feedback during processing
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey border */
                                     border-top: 12px solid #01cc97; /* Blue border */
                                     border-radius: 50%;             /* Circular shape */
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite; /* Spinning animation */'>
        </div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }     /* Start rotation */
        100% { transform: rotate(360deg); } /* End rotation */
    }
    </style>
    """

# HTML code to display a checkmark when loading is complete
done = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>  <!-- Checkmark symbol -->
    </div>
    """

# Create HTML widgets to display the loading animation and elapsed time
loader_html1 = widgets.HTML(value=loading)  # Initially show loading animation
timer_html1 = widgets.HTML(value="Elapsed time: - seconds")  # Placeholder for elapsed time

def generate_qam_constellation(L):
    """
    Generate the QAM constellation points for a square L x L QAM.

    Parameters:
    - L: Number of points along one axis (e.g., L=4 for 16-QAM)

    Returns:
    - mapping: 1D numpy array of complex numbers representing constellation points
    """
    M = L * L                  # Total number of constellation points
    l = int(np.log2(L))        # Number of bits per axis

    # Generate equally spaced points along the real and imaginary axes
    x = np.arange(-(L - 1), L, 2)
    y = np.arange(-(L - 1), L, 2)

    # Create a grid of points (constellation)
    xv, yv = np.meshgrid(x, y)
    mapping = xv + 1j * yv     # Combine real and imaginary parts
    mapping = mapping.flatten()  # Flatten to 1D array

    return mapping

def plot_qam_constellation(L):
    """
    Plot the QAM constellation for a given L.

    Parameters:
    - L: Number of points along one axis (e.g., L=4 for 16-QAM)
    """
    # Start timer and display loading animation
    loader_html1.value = loading
    start_time = time.time()

    mapping = generate_qam_constellation(L)
    M = L * L
    l = int(np.log2(L))

    # Plot the constellation points
    plt.figure(figsize=(10, 7))
    plt.scatter(mapping.real, mapping.imag)

    if L < 16:  # Include labels for smaller constellations
        # Generate binary labels for each point
        labels = [bin(i)[2:].zfill(2 * l) for i in range(M)]
        dx, dy = -0.5, 0.3  # Offset for labels
        for i in range(len(labels)):
            plt.text(mapping[i].real + dx, mapping[i].imag + dy, labels[i],
                     bbox=dict(facecolor='red', alpha=0.5))

    plt.grid(True)
    plt.xlim(-L, L)
    plt.ylim(-L, L)
    plt.title(f"{L}x{L} QAM Constellation")
    plt.xlabel("In-phase")
    plt.ylabel("Quadrature")
    plt.show()

    # Update elapsed time and loading animation
    elapsed_time = time.time() - start_time
    timer_html1.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html1.value = done

# Create a dropdown widget to select the value of L
L_dropdown = Dropdown(options=[2, 4, 8, 16, 32, 64], value=8, description='L-QAM:', continuous_update=False)

# Group the loader and timer widgets vertically
loader_timer_box = widgets.VBox([loader_html1, timer_html1], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create an interactive plot that updates when L changes
interactive_plot = interact(plot_qam_constellation, L=L_dropdown)

# Arrange input widgets and output
input_widgets = VBox([L_dropdown], layout=Layout(width='auto'))
plot_output = interactive_plot.widget.children[-1]  # The output plot widget

# Combine input widgets and loader/timer into a horizontal box
inputs_and_loader = HBox([input_widgets, loader_timer_box], layout=Layout(align_items='center'))

# Create a vertical box to hold the entire UI
ui = VBox([inputs_and_loader, plot_output])

# Clear previous output and display the UI
clear_output(wait=True)
display(ui)

In [6]:
# Loading animation HTML code for visual feedback during processing
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey border */
                                     border-top: 12px solid #01cc97; /* Blue border */
                                     border-radius: 50%;             /* Circular shape */
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite; /* Spinning animation */'>
        </div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }     /* Start rotation */
        100% { transform: rotate(360deg); } /* End rotation */
    }
    </style>
    """

# HTML code to display a checkmark when loading is complete
done = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>  <!-- Checkmark symbol -->
    </div>
    """

# Create HTML widgets to display the loading animation and elapsed time
loader_html2 = widgets.HTML(value=loading)
timer_html2 = widgets.HTML(value="Elapsed time: - seconds")

def generate_psk_constellation(M):
    """
    Generate the PSK constellation points for M-PSK modulation.

    Parameters:
    - M: Modulation order (e.g., M=8 for 8-PSK)

    Returns:
    - mapping: Numpy array of complex numbers representing constellation points
    """
    k = int(np.log2(M))  # Number of bits per symbol
    ph1 = np.pi / 4      # Initial phase

    # Initialize theta with initial phase values
    theta = np.array([ph1, -ph1, np.pi - ph1, -np.pi + ph1])
    mapping = np.exp(1j * theta)  # Initial constellation points

    # Generate PSK constellation for higher-order M
    if k > 2:
        for j in range(3, k + 1):
            theta = theta / 2
            mapping = np.exp(1j * theta)
            mapping = np.concatenate((mapping, -np.conjugate(mapping)))  # Mirror across origin
            theta = np.angle(mapping)  # Update theta

    return mapping

def plot_psk_constellation(M):
    """
    Plot the PSK constellation for a given M.

    Parameters:
    - M: Modulation order (e.g., M=8 for 8-PSK)
    """
    # Start timer and display loading animation
    loader_html2.value = loading
    start_time = time.time()

    constellation = generate_psk_constellation(M)
    k = int(np.log2(M))

    # Plot the constellation points
    plt.figure(figsize=(10, 7))
    plt.scatter(np.real(constellation), np.imag(constellation))
    plt.grid(True)
    plt.title(f'{M}-PSK Constellation')
    plt.xlabel('In-Phase')
    plt.ylabel('Quadrature')
    plt.axhline(0, color='gray', linewidth=0.5)  # Horizontal axis
    plt.axvline(0, color='gray', linewidth=0.5)  # Vertical axis

    # Label each point with its binary representation
    for m in range(len(constellation)):
        plt.text(np.real(constellation[m]) + 0.05, np.imag(constellation[m]),
                 format(m, '0{}b'.format(k)),
                 bbox=dict(facecolor='red', alpha=0.5))
    plt.show()

    # Update elapsed time and loading animation
    elapsed_time = time.time() - start_time
    timer_html2.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html2.value = done

# Create a dropdown widget to select the value of M
M_dropdown = widgets.Dropdown(options=[4, 8, 16, 32, 64], value=16, description='M-PSK:')

# Group the loader and timer widgets vertically
loader_timer_box = widgets.VBox([loader_html2, timer_html2], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create an interactive plot that updates when M changes
interactive_plot = interact(plot_psk_constellation, M=M_dropdown)

# Arrange input widgets and output
input_widgets = VBox([M_dropdown], layout=Layout(width='auto'))
plot_output = interactive_plot.widget.children[-1]

# Combine input widgets and loader/timer into a horizontal box
inputs_and_loader = HBox([input_widgets, loader_timer_box], layout=Layout(align_items='center'))

# Create a vertical box to hold the entire UI
ui = VBox([inputs_and_loader, plot_output])

# Clear previous output and display the UI
clear_output(wait=True)
display(ui)

## Part 2: Design and Simulation of M-QAM System
### Question 2
We have available the 6.75-9.25 MHz bandpass channel and want to transmit at a rate of 10 Mbps. Select a complete rectangular constellation and Nyquist signaling M-QAM system suitable for this purpose. Choose the smallest possible M and an appropriate roll-off value to take advantage of all available bandwidth. Simulate transmitter and receiver and plot the Pb vs Eb/No curve theoretically and experimentally. The sampling frequency must be sufficiently high so that the signals of all modulation-demodulation stages can be represented without aliasing.
    
### Connection with theory: 
The transmission rate, R (bits/s), is related to the symbol rate, 1/Τ (also called baud rate) and the size of the signal constellation, M, with the relation

$$
\frac{R}{\log_2 L} = \frac{l}{T} 
$$

On the other hand, the required bandwidth for bandpass transmission with Nyquist signaling is equal to

$$
W = \frac{1}{T}(1+\alpha)
$$

where α is the roll-off factor of the Nyquist filter used. Combining the above relations, we conclude that the size of the constellation must satisfy the inequality: 

$$
log_2 M >= \frac{R}{W}(1+\alpha), 0<a<=1
$$


```{image} ../content/images/Lab5_1.png
:align: center
```

In [7]:
# HTML loading animation and completion checkmark
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey border */
                                     border-top: 12px solid #01cc97; /* Blue border */
                                     border-radius: 50%;             /* Circular shape */
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite; /* Spinning animation */'>
        </div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }     /* Start rotation */
        100% { transform: rotate(360deg); } /* End rotation */
    }
    </style>
    """

done = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>  <!-- Checkmark symbol -->
    </div>
    """

# Create HTML widgets to display the loading animation and elapsed time
loader_html3 = widgets.HTML(value=loading)
timer_html3 = widgets.HTML(value="Elapsed time: - seconds")

# Function to calculate roll-off factor and recommend the best M
def calculate_rolloff(f1, f2, R):
    """
    Calculate the roll-off factor and recommend the best modulation order M.

    Parameters:
    - f1: Lower frequency in MHz
    - f2: Upper frequency in MHz
    - R: Data rate in Mbps

    Returns:
    - best_a: Calculated roll-off factor
    - best_M: Recommended modulation order M
    """
    W = (f2 - f1) * 1e6  # Bandwidth in Hz
    R = R * 1e6          # Data rate in bps
    possible_Ms = [4, 16, 64, 256, 1024, 4096, 16384]  # Possible M values
    best_M = None
    best_a = None
    for M in possible_Ms:
        a = np.log2(M) * W / R - 1
        if 0 < a < 1:
            best_M = M
            best_a = a
            break
    return best_a, best_M

# Input widgets for frequencies and data rate
f1_input = widgets.FloatText(description='F1 (MHz):', value=6.75)
f2_input = widgets.FloatText(description='F2 (MHz):', value=9.25)
R_input = widgets.FloatText(description='R (Mbps):', value=12)

# Function to update and display the result
def update_result(f1, f2, R):
    # Start timer and display loading animation
    loader_html3.value = loading
    start_time = time.time()

    a, M = calculate_rolloff(f1, f2, R)

    # Update elapsed time and loading animation
    elapsed_time = time.time() - start_time
    timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html3.value = done

    # Display the result
    if M is not None:
        result_html.value = f"Result: α = {a:.4f}, Recommended M = {M}"
    else:
        result_html.value = "No suitable M found within the given constraints."

# Widget to display the result
result_html = widgets.HTML(value="Result: α = 0.2500, Recommended M = 64")

# Group the loader and timer widgets vertically
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Arrange input widgets
input_widgets = widgets.VBox([f1_input, f2_input, R_input], layout=widgets.Layout(width='auto'))

# Combine inputs and loader/timer into a horizontal box
ui = widgets.HBox([input_widgets, loader_timer_box], layout=widgets.Layout(align_items='center'))

# Create interactive output
out = widgets.interactive_output(update_result, {'f1': f1_input, 'f2': f2_input, 'R': R_input})

# Clear previous output and display the UI
clear_output(wait=True)
display(ui, result_html)

HTML(value='Result: α = 0.2500, Recommended M = 64')

In [10]:
# Loading animation and completion checkmark HTML code
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """

done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """

# Create HTML widgets for loading animation and elapsed time
loader_html4 = widgets.HTML(
  value=loading
)
timer_html4 = widgets.HTML(
    value="Elapsed time: - seconds"
)

# HTML widget for displaying warnings
warning_html = widgets.HTML(
    value=""
)

def rootRaisedCosine(nsamp, roll_off, delay):
    """
    Generate a root raised cosine filter.

    Parameters:
    - nsamp: Number of samples per symbol
    - roll_off: Roll-off factor
    - delay: Filter delay in symbol periods

    Returns:
    - h: Filter coefficients
    """
    # Calculate necessary frequencies and times
    F0 = 0.5 / nsamp
    Br = 1
    Fs = Br * nsamp
    Td = 1 / Br
    Ts = 1 / Fs
    F1 = F0 * (1 - roll_off)
    F2 = F0 * (1 + roll_off)
    filter_order = 2 * nsamp * delay

    t = np.arange(0, filter_order, Td)
    h = []
    for i in range(len(t)):
        t_shifted = t[i] - filter_order / 2
        if t_shifted == 0:
            h.append(np.sqrt(2 * F0) *(1 + roll_off * ((4 / np.pi) - 1)))
        elif t_shifted == 1 / 8 / roll_off / F0 or t_shifted == - 1 / 8 / roll_off / F0 :
            h.append((roll_off * np.sqrt(F0)) * ((1 + 2 / np.pi) * np.sin(np.pi / 4 / roll_off) + (1 - 2 / np.pi) * np.cos(np.pi / 4 / roll_off)))
        else:
            factor1 = np.sqrt(2 * F0) / (1 - 64 * roll_off * roll_off * F0 * F0 * t_shifted * t_shifted)
            factor2 = np.sin(2 * np.pi * F1 * t_shifted) / (2 * np.pi * F0 * t_shifted)
            factor3 = (4 * roll_off / np.pi) * np.cos(2 * np.pi * F2 * t_shifted)
            h.append(factor1 * (factor2 + factor3))

    return h

def ber_qam(EbNo, M, roll_off, F1, F2, Br):
    """
    Calculate the Bit Error Rate (BER) for QAM modulation.

    Parameters:
    - EbNo: Energy per bit to noise power spectral density ratio in dB
    - M: Modulation order
    - roll_off: Roll-off factor
    - F1: Lower frequency in MHz
    - F2: Upper frequency in MHz
    - Br: Bit rate in Mbps

    Returns:
    - BER: Bit Error Rate
    """
    # Convert units from MHz and Mbps to Hz and bps
    F1 = F1 * 1e6
    F2 = F2 * 1e6
    Br = Br * 1e6
    W = F2 - F1         # Bandwidth in Hz
    fc = F1 + W / 2     # Carrier frequency
    nsamp = int(np.ceil(2 * F2 / Br)) +7  # Number of samples per symbol

    a = 0.25    # Roll-off factor
    L = int(np.sqrt(M))
    l = np.log2(L)
    k = 2 * l
    Nsymb = 10000
    SNR = EbNo - 10 * np.log10(nsamp / k / 2)  # σε db
    core = [1+1j, 1-1j, -1+1j, -1-1j]
    mapping = core[:]
    if l > 1:
        for j in range(1, int(l)):
            mapping = list(map(lambda x: x + j * 2 * core[0], mapping))
            conj_arr = np.conj(mapping)
            mapping = mapping + conj_arr.tolist()
            conj_arr = -np.conj(mapping)
            mapping = mapping + conj_arr.tolist()

    # Generate random sequence
    x = np.floor(2 * np.random.rand(int(k * Nsymb), 1))
    x_temp = np.reshape(x, (int(len(x) / (k)), int(k)))
    xsym = []

    # Split the list into sublists and put the contents of each sublist
    # into a string so that with the int() command it is converted from binary to decimal
    for i in range(len(x_temp)):
        my_str = ''
        y = x_temp[i]
        for j in range(int(np.log2(M))):
            my_str = my_str + str(int(y[j]))
        a = int(my_str, 2)
        xsym = xsym + [a]

    y = []
    for n in range(len(xsym)):
        y = y + [mapping[xsym[n]]]

    # Generate shaping filter
    delay = 10
    filtorder = delay * nsamp * 2
    shaping_filter = rootRaisedCosine(nsamp, roll_off, delay)
    
    # Transmit signal
    ytx = upfirdn([1], y, nsamp)  # upsample
    ytx = np.convolve(ytx, shaping_filter)
    m = np.arange(1, len(ytx) + 1)
    s = np.real(np.multiply(ytx, np.exp(1j * 2 * np.pi * fc * m / nsamp)))

    s_matrix = np.matrix(s)  # transpose
    s_matrix = s_matrix.getH()
    s_list = s_matrix.tolist()
    Ps = 10 * np.log10(np.matmul(s, s_list) / len(s))  # Power of complex signal in dB
    Pn = Ps - SNR

    n = np.sqrt(10**(Pn / 10)) * np.random.randn(1, len(ytx))
    snoisy = s + n

    # receiver
    yrx = 2 * np.multiply(snoisy, np.exp(-1j * 2 * np.pi * fc * m / nsamp))
    yrx = yrx[0, :]
    yrx = np.convolve(yrx, shaping_filter)
    yrx = yrx[::nsamp]  # downsample

    yrx = yrx[2 * delay + 0:len(yrx) - 2 * delay]

    yi = yrx.copy()
    yq = np.imag(yi)
    yi = np.real(yi)

    xrx = []
    q = np.arange(-L + 1, L, 2)

    for n in range(len(yrx)):
        differences = np.abs(q - yi[n])  # Array with the differences of the signal from the levels
        m = min(differences)
        [index], = np.where(differences == m)
        yi[n] = q[index]
        differences = np.abs(q - yq[n])  # Array with the differences of the signal from the levels
        m = min(differences)
        [index], = np.where(differences == m)
        yq[n] = q[index]
    error = 0
    for i in range(len(yrx)):
        if y[i] != yi[i] + yq[i] * 1j:
            error += 1 
    return error / len(x)

def plot_ber_qam(M, roll_off, F1, F2, Br):
    """
    Plot the BER curve for QAM modulation.

    Parameters:
    - M: Modulation order
    - roll_off: Roll-off factor
    - F1: Lower frequency in MHz
    - F2: Upper frequency in MHz
    - Br: Bit rate in Mbps
    """
    # Validate input frequencies
    if F1 >= F2:
        warning_html.value = "<div style='color: red; font-size: 16px;'>Warning: F1 should be less than F2.</div>"
        return
    
    warning_html.value = ""  # Clear any previous warnings
    
    # Start the timer
    loader_html4.value = loading
    start_time = time.time()

    ber_exp = []
    ber_th = []
    L = int(np.sqrt(M))
    for i in range(1, 15):
        ber_exp.append(ber_qam(i, M, roll_off, F1, F2, Br))
        ber_th.append(((L - 1) /(L*np.log2(L)) * scipy.special.erfc(np.sqrt(3 * np.log2(L) / (L * L - 1) * 10**(i/10)))))

    plt.figure(figsize=(10, 8))
    plt.semilogy(range(1, 15), ber_th)  # Plot theoretical BER as a line
    plt.semilogy(range(1, 15), ber_exp, 'o')  # Plot experimental BER as points
    plt.legend(['Theoretical', 'Simulation'])
    plt.xlabel('Eb/N0(db)')
    plt.ylabel('Bit Error Probability')
    plt.title(f'BER curve for {M}-QAM')
    plt.grid(which='both')
    plt.show()

    # Stop the timer and update the timer HTML
    elapsed_time = time.time() - start_time
    timer_html4.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    # Update the loading animation to done
    loader_html4.value = done

# Define QAM options
qam_options = {'4-QAM': 4, '16-QAM': 16, '64-QAM': 64}
qam_selector = widgets.Dropdown(options=qam_options, value=16, description='QAM Type:')

# Define additional input boxes for roll-off, F1, F2, and Br
roll_off_input = widgets.FloatText(value=0.25, description='Roll-off:')
F1_input = widgets.FloatText(value=6.75, description='F1: (MHz)')
F2_input = widgets.FloatText(value=9.25, description='F2: (MHz)')
Br_input = widgets.FloatText(value=10, description='Br: (Mbps)')

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html4, timer_html4], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create a VBox for the input widgets (similar to the first code snippet)
input_widgets = widgets.VBox([qam_selector, roll_off_input, F1_input, F2_input, Br_input, warning_html], layout=widgets.Layout(width='auto'))

# Create an HBox to combine inputs and loader timer box, with the same layout style
ui = widgets.HBox([input_widgets, loader_timer_box], layout=widgets.Layout(align_items='center'))

# Create the interactive output for the plot_ber_qam function
out = widgets.interactive_output(plot_ber_qam, {'M': qam_selector, 'roll_off': roll_off_input, 'F1': F1_input, 'F2': F2_input, 'Br': Br_input})

# Display the UI and output
clear_output(wait=True)  # Clear the previous output
display(ui, out)

Output()

## Part 3: Adaptive QAM System
### Question 3
If, for the system in question 2, the maximum normalized signal-to-noise ratio, Eb/No, that you can achieve at the receiver, is 10 db, and the channel encoder you have requires that the bit error probability does not exceed 0.005, fall back into a lower-order, rectangular grid QAM system again, without changing the other signaling parameters. What is the maximum baud rate now? Plot the power spectrum density of your signals again and see if there are any differences. Examine if a less fall back in a non-rectangular grid system could
satisfy the required transmission quality.

In [11]:
# Loading animation and done checkmark HTML code
# These HTML snippets are used to display a loading spinner and a completion checkmark in the UI.

loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey border */
                                     border-top: 12px solid #01cc97; /* Greenish-blue border on top */
                                     border-radius: 50%;             /* Make it a circle */
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite; /* Spin animation */'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }     /* Start at 0 degrees */
        100% { transform: rotate(360deg); } /* Rotate full circle */
    }
    </style>
    """

done = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>  <!-- Unicode checkmark -->
    </div>
    """

# Create HTML widgets for the loading animation and timer display
loader_html5 = widgets.HTML(value=loading)
timer_html5 = widgets.HTML(value="Elapsed time: - seconds")

def rtrapezium(nsamp, rolloff, delay):
    """
    Generate a root-raised cosine (RRC) filter using the trapezium method.

    Parameters:
    - nsamp (int): Number of samples per symbol (oversampling factor)
    - rolloff (float): Roll-off factor (0 < rolloff <= 1)
    - delay (int): Filter delay in symbol periods

    Returns:
    - rrc (numpy array): Filter coefficients
    """
    T = 1  # Symbol period
    t = np.arange(-delay*T, (delay*T) + 1/nsamp, 1/nsamp)  # Time vector
    rrc = np.zeros_like(t)  # Initialize filter coefficients

    # Calculate filter coefficients
    for i, ti in enumerate(t):
        if ti == 0.0:
            rrc[i] = 1.0 - rolloff + 4 * rolloff / np.pi
        elif abs(ti) == T / (4 * rolloff):
            rrc[i] = (rolloff / np.sqrt(2)) * (((1 + 2/np.pi) * np.sin(np.pi / (4 * rolloff))) +
                                               ((1 - 2/np.pi) * np.cos(np.pi / (4 * rolloff))))
        else:
            numerator = (np.sin(np.pi * ti * (1 - rolloff) / T) +
                         4 * rolloff * ti * np.cos(np.pi * ti * (1 + rolloff) / T) / T)
            denominator = (np.pi * ti * (1 - (4 * rolloff * ti / T) ** 2))
            rrc[i] = numerator / denominator

    # Normalize filter coefficients
    rrc = rrc / np.sqrt(np.sum(rrc**2))
    return rrc

def run_simulation(f1, f2, qam_type):
    """
    Simulate the transmission and reception of a QAM signal and plot the power spectral density.

    Parameters:
    - f1 (float): Lower cutoff frequency in MHz
    - f2 (float): Upper cutoff frequency in MHz
    - qam_type (int): Modulation order (e.g., 16 for 16-QAM)
    """
    # Start the timer and display loading animation
    loader_html5.value = loading
    start_time = time.time()

    # Parameters
    k = int(np.log2(qam_type))   # Bits per symbol
    M = 2**k                     # Modulation order
    Nsymb = 30000                # Number of symbols
    pulse_type = 1               # Pulse shaping filter type (1 for RRC)
    nsamp = 32                   # Oversampling factor
    fc = (f1 + f2) / 2           # Carrier frequency (MHz)
    bandwidth = f2 - f1          # Signal bandwidth (MHz)
    rolloff = bandwidth / (2 * fc)  # Roll-off factor
    EbNo = 10                    # Energy per bit to noise power spectral density ratio in dB
    SNR = EbNo - 10 * np.log10(nsamp / k / 2)  # Signal-to-noise ratio per sample

    # Phase and mapping initialization
    ph1 = np.pi / 4
    theta = np.array([ph1, -ph1, np.pi - ph1, -np.pi + ph1])
    mapping = np.exp(1j * theta)

    if k > 2:
        # Generate PSK constellation for higher-order modulation
        for j in range(3, k + 1):
            theta = theta / 2
            mapping = np.exp(1j * theta)
            mapping = np.concatenate([mapping, -np.conj(mapping)])
            theta = np.angle(mapping)

    # Transmitter
    x = np.random.randint(0, 2, k * Nsymb)  # Random binary sequence
    xsym = x.reshape(-1, k)
    xsym = xsym.dot(2**np.arange(xsym.shape[-1])[::-1])  # Convert bits to decimal symbols
    y = mapping[xsym]  # Map symbols to constellation points

    # Shaping filter definition
    if pulse_type == 1:  # Nyquist pulse shaping with RRC filter
        delay = 8  # Group delay in symbol periods
        shaping_filter = rtrapezium(nsamp, rolloff, delay)
    else:  # Rectangular pulse
        delay = 0.5
        shaping_filter = np.ones(nsamp) / np.sqrt(nsamp)  # Normalize filter

    # Transmitted signal
    ytx = upfirdn([1], y, nsamp)  # Upsample the signal
    ytx = convolve(ytx, shaping_filter, mode='same')  # Apply shaping filter

    # Quadrature modulation
    m = np.arange(len(ytx))
    s = np.real(ytx * np.exp(1j * 2 * np.pi * fc * m / nsamp))  # Modulate to carrier frequency

    # Adding white Gaussian noise
    Ps = 10 * np.log10(np.mean(s**2))  # Signal power in dB
    Pn = Ps - SNR                      # Noise power in dB
    n = np.sqrt(10**(Pn / 10)) * np.random.randn(len(ytx))  # Generate noise
    snoisy = s + n  # Noisy bandpass signal

    # Receiver
    yrx = 2 * snoisy * np.exp(-1j * 2 * np.pi * fc * m / nsamp)  # Demodulate
    yrx = convolve(yrx, shaping_filter, mode='same')             # Apply matched filter

    # Spectrum plot of received signal
    f, Pxx_den = welch(np.real(s), fs=nsamp, nperseg=1024)
    Pxx_den = 10 * np.log10(Pxx_den)  # Convert power spectral density to dB
    plt.figure(figsize=(10, 8))
    plt.plot(f, Pxx_den, 'r')
    plt.title('Welch Power Spectral Density Estimate')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power Spectral Density (dB/Hz)')
    plt.xlim(0, max(f1, f2) + 5)
    plt.grid()
    plt.show()

    # Stop the timer and update the loading animation
    elapsed_time = time.time() - start_time
    timer_html5.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html5.value = done

# Widgets for input parameters
f1_widget = widgets.FloatText(value=6.75, description='f1 (MHz):')
f2_widget = widgets.FloatText(value=9.25, description='f2 (MHz):')
qam_widget = widgets.Dropdown(options=[4, 16, 64], value=16, description='QAM Type:')

# Create the interactive output for the run_simulation function
out = widgets.interactive_output(run_simulation, {'f1': f1_widget, 'f2': f2_widget, 'qam_type': qam_widget})

# Group the loader and timer together vertically
loader_timer_box = widgets.VBox([loader_html5, timer_html5], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Arrange input widgets in a vertical box
input_widgets = widgets.VBox([qam_widget, f1_widget, f2_widget], layout=widgets.Layout(width='auto'))

# Combine input widgets and loader/timer into a horizontal box
ui = widgets.HBox([input_widgets, loader_timer_box], layout=widgets.Layout(align_items='center'))

# Display the UI and output
display(ui, out)

Output()

### Question 4
How much can the bit rate in question 3 be increased if the roll-off of the Nyquist filter can be reduced by 50%?

In [12]:
# Loading animation and done checkmark HTML code
# These HTML snippets are used to display a loading spinner and a completion checkmark in the UI.

loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey border */
                                     border-top: 12px solid #01cc97; /* Greenish-blue border on top */
                                     border-radius: 50%;             /* Make it a circle */
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite; /* Spin animation */'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }     /* Start at 0 degrees */
        100% { transform: rotate(360deg); } /* Rotate full circle */
    }
    </style>
    """

done = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>  <!-- Unicode checkmark -->
    </div>
    """

loader_html6 = widgets.HTML(value=loading)
timer_html6 = widgets.HTML(value="Elapsed time: - seconds")

# Function to calculate the maximum achievable transmission rate R' and the percentage increase
def calculate_R_and_percentage_increase(M, a, R):
    """
    Calculate the maximum achievable rate R' and the percentage increase over the current rate R.

    Parameters:
    - M (float): Modulation order
    - a (float): Roll-off factor
    - R (float): Current data rate in Mbps

    Returns:
    - R_prime_mbps (float): Maximum achievable rate in Mbps
    - percentage_increase (float): Percentage increase over the current rate
    """
    W = 2.5 * 1e6  # Fixed bandwidth in Hz
    log2M = np.log2(M)
    R_prime = (log2M * W) / (1 + a)  # Maximum achievable rate in bps
    R_prime_mbps = R_prime / 1e6     # Convert to Mbps
    percentage_increase = ((R_prime_mbps - R) / R) * 100  # Percentage increase
    return R_prime_mbps, percentage_increase

# Input widgets for R, M, and a
R_input = widgets.FloatText(description='R (Mbps):', value=8.0)
M_input = widgets.FloatText(description='M:', value=16)
a_input = widgets.FloatText(description='α (Roll-off):', value=0.125)

# Output widget for displaying results
output_vals = widgets.Output()

# Function to update the result when any input changes
def update_result(change=None):
    # Start the timer and display loading animation
    loader_html6.value = loading
    start_time = time.time()

    with output_vals:
        output_vals.clear_output()
        R = R_input.value
        M = M_input.value
        a = a_input.value
        R_prime_mbps, percentage_increase = calculate_R_and_percentage_increase(M, a, R)
        print(f"Maximum Achievable Rate (R') = {R_prime_mbps:.3f} Mbps")
        print(f"Percentage Increase = {percentage_increase:.2f}%")

        # Stop the timer and update the loading animation
        elapsed_time = time.time() - start_time
        timer_html6.value = f"Elapsed time: {elapsed_time:.2f} seconds"
        loader_html6.value = done

# Attach observers to input widgets to trigger result update
R_input.observe(update_result, names='value')
M_input.observe(update_result, names='value')
a_input.observe(update_result, names='value')

# Group the loader and timer together vertically
loader_timer_box = widgets.VBox([loader_html6, timer_html6], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Arrange input widgets in a vertical box
input_widgets = VBox([R_input, M_input, a_input], layout=Layout(width='auto'))

# Combine input widgets and loader/timer into a horizontal box
inputs_and_loader = HBox([input_widgets, loader_timer_box], layout=Layout(align_items='center'))

# Create a vertical box that includes both inputs and output
ui = VBox([inputs_and_loader, output_vals])

# Clear previous output and display the UI
clear_output(wait=True)
display(ui)

# Initial calculation to display results
update_result()

### Question 5
(Optional) Simulate a PSK system, of the same modulation order and with the same roll-off factor of Nyquist signals, as the QAM of question 3. Compare with the latter 
1. in terms of BER, 
2. in terms of bandwidth, after designing the BER-EbNo curve and the spectrum of the generated signal. 

Gray coding should also be used here, implementing the procedure described in Frame 5.10 of the Course Notes, with the help of the code:

``````{dropdown} Code

`````{tab} Python
````python
# The mapping vector for M-QAM Gray coding
# It is for a complete rectangular constellation, of dimension M=L^2
# l=log2(L): number of bits per dimension (inphase, quadrature)
core = np.array([1+1j, 1-1j, -1+1j, -1-1j]) # trivial case, M=4
mapping = core

if (l > 1):
    for k in range (1,l):
        mapping = mapping + k*2*core[0]
        mapping = np.array([mapping, np.conj(mapping)])
        mapping = np.array([mapping, (-np.conj(mapping))])
        
mapping = mapping.flatten().T

````
`````
`````{tab} Matlab
````matlab
% The mapping vector for M-PSK Gray coding
k=log2(M);
ph1=[pi/4];
theta=[ph1; -ph1; pi-ph1; -pi+ph1];
mapping=exp(1j*theta);
if(k>2)
 for j=3:k
 theta=theta/2;
 mapping=exp(1j*theta);
 mapping=[mapping; -conj(mapping)];
 theta=real(log(mapping)/1j);
 end
end

````
`````

``````

In [16]:
# HTML for loading animation and completion checkmark
# These HTML snippets are used to display a loading spinner and a completion checkmark in the UI.

loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey border */
                                     border-top: 12px solid #01cc97; /* Greenish-blue border on top */
                                     border-radius: 50%;             /* Make it a circle */
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite; /* Spin animation */'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }     /* Start at 0 degrees */
        100% { transform: rotate(360deg); } /* Rotate full circle */
    }
    </style>
    """

done = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>  <!-- Unicode checkmark -->
    </div>
    """

loader_html7 = widgets.HTML(value=loading)
timer_html7 = widgets.HTML(value="Elapsed time: - seconds")

# Define the root raised cosine filter function
def rootRaisedCosine1(nsamp, roll_off, delay):
    """
    Generate a root raised cosine (RRC) filter.

    Parameters:
    - nsamp (int): Number of samples per symbol
    - roll_off (float): Roll-off factor
    - delay (int): Filter delay in symbol periods

    Returns:
    - h (numpy array): Filter coefficients
    """
    t = np.arange(-delay, delay + 1 / nsamp, 1 / nsamp)
    h = np.zeros(len(t))
    for i in range(len(t)):
        if t[i] == 0.0:
            h[i] = 1.0 - roll_off + 4 * roll_off / np.pi
        elif roll_off != 0 and t[i] == 1 / (4 * roll_off):
            h[i] = roll_off / np.sqrt(2) * ((1 + 2 / np.pi) * np.sin(np.pi / (4 * roll_off)) + (1 - 2 / np.pi) * np.cos(np.pi / (4 * roll_off)))
        elif roll_off != 0 and t[i] == -1 / (4 * roll_off):
            h[i] = roll_off / np.sqrt(2) * ((1 + 2 / np.pi) * np.sin(np.pi / (4 * roll_off)) + (1 - 2 / np.pi) * np.cos(np.pi / (4 * roll_off)))
        else:
            h[i] = (np.sin(np.pi * t[i] * (1 - roll_off)) + 4 * roll_off * t[i] * np.cos(np.pi * t[i] * (1 + roll_off))) / (np.pi * t[i] * (1 - (4 * roll_off * t[i]) ** 2))
    return h

# Define the BER computation functions for PSK modulation
def compute_ber_psk(EbNo_dB, M1):
    """
    Compute the theoretical Bit Error Rate (BER) for M-PSK modulation.

    Parameters:
    - EbNo_dB (float): Eb/N0 ratio in dB
    - M1 (int): Modulation order (e.g., 2 for BPSK, 4 for QPSK)

    Returns:
    - BER (float): Bit Error Rate
    """
    EbNo_linear = 10**(EbNo_dB / 10)
    if M1 == 2: # BPSK
        return 0.5 * scipy.special.erfc(np.sqrt(EbNo_linear))
    else:       # M-PSK
        k = np.log2(M1)
        return (1/4*k) * scipy.special.erfc(np.sqrt(EbNo_linear * k) * np.sin(np.pi / M1))

def ber_psk_simulation(EbNo_dB, M1):
    """
    Simulate the Bit Error Rate (BER) for M-PSK modulation over an AWGN channel.

    Parameters:
    - EbNo_dB (float): Eb/N0 ratio in dB
    - M1 (int): Modulation order

    Returns:
    - ber (float): Bit Error Rate from simulation
    """
    Nsymb = 30000   # Number of symbols
    nsamp = 16      # Samples per symbol
    fc = 4          # Carrier frequency
    rolloff = 0.25
    delay = 10
    SNR_dB = EbNo_dB - 10 * np.log10(nsamp / np.log2(M1))
    shaping_filter = rootRaisedCosine1(nsamp, rolloff, delay)
    filtorder = delay * nsamp * 2

    # Generate random symbol sequence
    bits1 = np.random.randint(0, M1, Nsymb)

    # Map symbols to PSK constellation points
    symbols = np.exp(1j * (2 * np.pi * bits1 / M1))

    # Upsample and filter the signal
    ytx1 = upfirdn([1], symbols, nsamp)
    ytx1 = np.convolve(ytx1, shaping_filter, mode='same')
    m1 = np.arange(len(ytx1))
    s1 = np.real(ytx1 * np.exp(1j * 2 * np.pi * fc * m1 / nsamp))

    # Calculate signal and noise power
    Ps = np.mean(np.abs(s1)**2)
    SNR_linear = 10**(SNR_dB / 10)
    Pn = Ps / SNR_linear

    # Generate AWGN noise
    noise = np.sqrt(Pn / 4) * (np.random.randn(len(s1)) + 1j * np.random.randn(len(s1)))
    snoisy = s1 + noise

    # Receiver
    yrx1 = snoisy * np.exp(-1j * 2 * np.pi * fc * m1 / nsamp)
    yrx1 = np.convolve(yrx1, shaping_filter, mode='same')
    yrx1 = yrx1[::nsamp]

    # Demodulate symbols
    detected_bits1 = np.angle(yrx1) * M1 / (2 * np.pi)
    detected_bits1 = np.round(detected_bits1) % M1

    # Calculate BER
    bit_errors1 = np.sum(bits1 != detected_bits1)
    ber1 = bit_errors1 / len(bits1)
    return ber1

def plot_ber_psk(M):
    """
    Plot the BER curve for M-PSK modulation.

    Parameters:
    - M (int): Modulation order (e.g., 2 for BPSK, 4 for QPSK)
    """
    # Start the timer and display loading animation
    loader_html7.value = loading
    start_time = time.time()

    ber_exp = []  # Experimental BER
    ber_th = []   # Theoretical BER
    for i in range(1, 18):
        ber_exp.append(ber_psk_simulation(i, M))
        ber_th.append(compute_ber_psk(i, M))

    # Plot BER curves
    plt.figure(figsize=(10, 8))
    plt.semilogy(range(1, 18), ber_th)  # Plot theoretical BER as a line
    plt.semilogy(range(1, 18), ber_exp, 'o')  # Plot experimental BER as points
    plt.legend(['Theoretical', 'Simulation'])
    plt.xlabel('Eb/N0 (dB)')
    plt.ylabel('Bit Error Probability')
    plt.title(f'BER curve for {M}-PSK')
    plt.grid(which='both')
    plt.show()

    # Stop the timer and update the timer HTML
    elapsed_time = time.time() - start_time
    timer_html7.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    # Update the loading animation to done
    loader_html7.value = done

# Define PSK options for the dropdown menu
psk_options = {'BPSK': 2, 'QPSK': 4, '8-PSK': 8}
psk_selector = widgets.Dropdown(options=psk_options, value=4, description='PSK Type:')

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html7, timer_html7], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create a VBox for the input widgets (similar to the first code snippet)
input_widgets = widgets.VBox([psk_selector], layout=widgets.Layout(width='auto'))

# Create an HBox to combine inputs and loader timer box, with the same layout style
ui = widgets.HBox([input_widgets, loader_timer_box], layout=widgets.Layout(align_items='center'))

# Create the interactive output for the plot_ber_psk function
out = widgets.interactive_output(plot_ber_psk, {'M': psk_selector})

# Display the UI and output
clear_output(wait=True)  # Clear the previous output
display(ui, out)

Output()